# A case study on Class Decorators

## Classic Inheritance

In [130]:
class MyParent():
    def __init__(self):
        print("MyParent:__init__")

    def __call__(self):
        print("MyParent:__call__")

    def my_method(self):
        print("MyParent:my_method")

    def my_parent_method(self):
        print("MyParent:my_parent_method")


class MyChild(MyParent):
    def __init__(self):
        print("MyChild:__init__")
        MyParent.__init__(self)

    def __call__(self):
        print("MyChild:__call__")

    def my_method(self):
        print("MyChild:my_method")

    def my_child_method(self):
        print("MyChild:my_child_method")

In [131]:
obj = MyChild()

MyChild:__init__
MyParent:__init__


Note how the classes are not initialized until called upon, and how MyParent is initialized with MyChild.
Also note that `__call__` is not executed, this is because we only initialized the object.

In [132]:
obj.my_method()
obj.my_parent_method()
obj.my_child_method()

MyChild:my_method
MyParent:my_parent_method
MyChild:my_child_method


As expected from inheritance, `my_method` of `obj` was inherited by `MyParent` but overwritten by `MyChild`. The `my_parent_method` works as it was inherited, and `my_child_method` also works as it comes directly from `MyChild`.

## Class Decorator

In [133]:
class MyParent():
    def __init__(self, Child):
        print("MyParent:__init__")

    def __call__(self):
        print("MyParent:__call__")

    def my_method(self):
        print("MyParent:my_method")

    def my_parent_method(self):
        print("MyParent:my_parent_method")

@MyParent
class MyChild(MyParent):
    def __init__(self):
        print("MyChild:__init__")
        MyParent.__init__(self)

    def __call__(self):
        print("MyChild:__call__")

    def my_method(self):
        print("MyChild:my_method")

    def my_child_method(self):
        print("MyChild:my_child_method")

MyParent:__init__


In [134]:
obj = MyChild()

MyParent:__call__


As you may have noticed, `MyParent` is initialized right away, meaning a `MyParent` object is created when the decorator is executed.
`MyChild` now runs the `__call___` from `MyParent`, and a `MyChild` is never initialized.

In [135]:
obj.my_method()

AttributeError: 'NoneType' object has no attribute 'my_method'

Since nothing was initilized, the obj variable is not an object, due to the `__call__` returning `None`.  
Therefore, none of our methods exist. Let's explore this further.

## Inverted Inheritance? (False Positive)

In [ ]:
class MyParent():
    def __init__(self, Child):
        print("MyParent:__init__")
        self.child = Child

    def __call__(self):
        print("MyParent:__call__")
        self.child = self.child()
        return self

    def my_method(self):
        print("MyParent:my_method")

    def my_parent_method(self):
        print("MyParent:my_parent_method")

@MyParent
class MyChild(MyParent):
    def __init__(self):
        print("MyChild:__init__")

    def __call__(self):
        print("MyChild:__call__")

    def my_method(self):
        print("MyChild:my_method")

    def my_child_method(self):
        print("MyChild:my_child_method")

MyParent:__init__


In [ ]:
obj = MyChild()

MyParent:__call__
MyChild:__init__


In [ ]:
obj.my_method()
obj.my_parent_method()
obj.child.my_child_method()
obj.my_child_method()

MyParent:my_method
MyParent:my_parent_method
MyChild:my_child_method


AttributeError: 'MyParent' object has no attribute 'my_child_method'

While this may appear like we have reversed the inheritance, we actually simply nested our objects.  
In this scenario, `MyClass` is initialized inside of `MyParent` but not inherited. In order to reach `MyClass` methods, the `child` object must be referenced.

## Inverted Inheritance

In [ ]:
def redirect_inheritance(MyClass):
    print("main:redirect_inheritance")
    class MyParent(MyClass):
        def __init__(self):
            print("MyParent:__init__")
            MyClass.__init__(self)

        def my_method(self):
            print("MyParent:my_method")

        def my_parent_method(self):
            print("MyParent:my_parent_method")
    return MyParent # IMPORTANT: Return the class, NOT an object of the class

@redirect_inheritance
class MyChild(MyParent):
    def __init__(self):
        print("MyChild:__init__")

    def __call__(self):
        print("MyChild:__call__")

    def my_method(self):
        print("MyChild:my_method")

    def my_child_method(self):
        print("MyChild:my_child_method")

main:redirect_inheritance


In [ ]:
obj = MyChild()

MyParent:__init__
MyChild:__init__


In [ ]:
obj.my_method()
obj.my_parent_method()
obj.my_child_method()

MyParent:my_method
MyParent:my_parent_method
MyChild:my_child_method


Let's look at what we've just done. When `MyChild` is called upon, it doesn't initialize `MyChild` first, but instead it initialized `MyParent`. Why?  
Well the decorator redirects our call to `MyChild` into a function which takes the `MyChild` class as an argument. This can then be passed into `MyParent`, officially flipping the inheritance logic. We then define `MyParent` and return the `MyParent` class.

As a result, the `MyParent` class is initialized first, followed by `MyChild`. Our methods behave as expected, inverted, where `my_method` is overwritten by `MyParent` now, and we still maintain individual methods like before.

## What is the value of this?

It's fair to say that if you have full access to your code, there are seldom (if any) scenario where this is useful. However, let's play out a scenario where you may not have acces to code. For example, you rely on someone else's code, and you don't want to modify it.

In [ ]:
# Cannot Modify
class TheirParent:
    def __init__(self):
        print("TheirParent:__init__")

    def __call__(self):
        print("TheirParent:__call__")

    def their_method(self):
        print("TheirParent:my_method")

    def their_parent_method(self):
        print("TheirParent:my_child_method")

# Can modify
class TheirChild(TheirParent):
    def __init__(self):
        print("TheirChild:__init__")

    def __call__(self):
        print("TheirChild:__call__")

    def their_method(self):
        print("TheirChild:my_method")

    def their_child_method(self):
        print("TheirChild:my_child_method")

# Cannot Modify
obj = TheirChild()

TheirChild:__init__


Imagine this is the code you're working with, but you want to interject with your own additions or modifications without completely undoing `TheirClass`. You may want to do this if `TheirClass` is in active development by another developer that is unaware of your presence.  
How would you do it?

- Add `MyClass` as a parent to `TheirChild`  
This is limiting, since the inheritance is backwards and you cannot modify their existing methods, you ocan only append to `TheirClass`.

- Modify the `class TheirChild` line with a different name, and add a new class with the original `TheirChild` name below the original, inheriting from it.  
This would work, but you've now modified their code. What if `TheirChild` is modified (i.e. parameters are added), then a conflict occurs.

- Invert the Inheritance (w/ decorators)
This solutions is clean, as it only *adds* 1 line of code. Below is our final example.

In [ ]:

def redirect_inheritance(TheirChild):
    print("main:redirect_inheritance")
    class MyParent(TheirChild):
            def __init__(self):
                print("MyParent:__init__")
                TheirChild.__init__(self)

            def my_method(self):
                print("MyParent:my_method")

            def my_parent_method(self):
                print("MyParent:my_parent_method")

            def modify_their_child_variable(self):
                self.their_child_variable = "new"

    return MyParent


In [ ]:
# Cannot Modify
class TheirParent:
    def __init__(self):
        print("TheirParent:__init__")
        self.their_parent_variable = "original"

    def __call__(self):
        print("TheirParent:__call__")

    def their_method(self):
        print("TheirParent:my_method")

    def their_parent_method(self):
        print("TheirParent:my_child_method")

# Can modify
@redirect_inheritance # Only line added/Modified
class TheirChild(TheirParent):
    def __init__(self):
        print("TheirChild:__init__")
        super().__init__()
        self.their_child_variable = "original"

    def __call__(self):
        print("TheirChild:__call__")

    def their_method(self):
        print("TheirChild:my_method")

    def their_child_method(self):
        print("TheirChild:my_child_method")

# Cannot Modify
obj = TheirChild()

main:redirect_inheritance
MyParent:__init__
TheirChild:__init__
TheirParent:__init__


In [ ]:
obj.their_method()
print(obj.their_parent_variable)
print(obj.their_child_variable)
obj.modify_their_child_variable()
print(obj.their_child_variable)

TheirChild:my_method
original
original
new
